In [1]:
import os
import torch
from random import randint
from utils.loss_utils import l1_loss, ssim
from gaussian_renderer import render, network_gui
import sys
from scene import Scene, GaussianModel
from utils.general_utils import safe_state
import uuid
from tqdm import tqdm
from utils.image_utils import psnr
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams

/scratch/cvlab/home/javed/anaconda3/envs/c3dgs/lib/python3.11/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:462: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/scratch/cvlab/home/javed/anaconda3/envs/c3dgs/lib/python3.11/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:647: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/scratch/cvlab/home/javed/anaconda3/envs/c3dgs/lib/python3.11/site-packages/vector_quantize_pytorch/finite_scalar_quantization.py:162: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


In [2]:
parser = ArgumentParser(description="Training script parameters")
lp = ModelParams(parser)
op = OptimizationParams(parser)
pp = PipelineParams(parser)
parser.add_argument('--ip', type=str, default="127.0.0.1")
parser.add_argument('--port', type=int, default=6009)
parser.add_argument('--debug_from', type=int, default=-1)
parser.add_argument('--detect_anomaly', action='store_true', default=False)
parser.add_argument("--test_iterations", nargs="+", type=int, default=[30_000])
parser.add_argument("--save_iterations", nargs="+", type=int, default=[30_000])
parser.add_argument("--quiet", action="store_true")
parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[])
parser.add_argument("--start_checkpoint", type=str, default = None)
parser.add_argument("--comp", action="store_true")
parser.add_argument("--store_npz", action="store_true")

args = parser.parse_args(["-s", "/scratch/cvlab/datasets/datasets_ahmad/mipNerf360/bicycle/", "-m", "output/bicycle_no_mlp_n_64_l_2", "--mlp_neurons", "64", "--mlp_layers", "2", "--use_trad_shs", "--post_training_mlp"])
args.save_iterations.append(args.iterations)

print("Optimizing " + args.model_path)

# Initialize system state (RNG)
safe_state(args.quiet)

# Start GUI server, configure and run training
network_gui.init(args.ip, args.port)
torch.autograd.set_detect_anomaly(args.detect_anomaly)

Optimizing output/bicycle_no_mlp_n_64_l_2


Loading trained model at iteration 30000 [20/09 13:49:22]
Reading camera 194/194 [20/09 13:49:23]
Loading Training Cameras [20/09 13:49:23]
[ INFO ] Encountered quite large input images (>1.6K pixels width), rescaling to 1.6K.
 If this is not desired, please explicitly specify '--resolution/-r' as 1 [20/09 13:49:23]
Loading Test Cameras [20/09 13:50:43]
Loading  output/bicycle_no_mlp_n_64_l_2/point_cloud/iteration_30000/point_cloud.ply [20/09 13:50:43]
Loading  output/bicycle_no_mlp_n_64_l_2.ply [20/09 13:54:16]
Loading  output/bicycle_no_mlp_n_64_l_2/point_cloud/iteration_30000.ply [20/09 13:55:24]
Loading  output/bicycle_no_mlp_n_64_l_2/point_cloud/iteration_30000/point_cloud.ply [20/09 13:55:34]


In [3]:
dataset = lp.extract(args)
opt = op.extract(args)
pipe = pp.extract(args)

In [4]:
gaussians = GaussianModel(dataset)
scene = Scene(dataset, gaussians, load_iteration="30000", shuffle=False)
gaussians.training_setup(opt)

/scratch/cvlab/home/ajkhan/Compact-3DGS/scene/gaussian_model.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.nn.ModuleList([self.recolor, self.mlp_head]).load_st

In [5]:
type(gaussians)

scene.gaussian_model.GaussianModel

In [12]:
gaussians.get_features.shape

torch.Size([2578590, 1, 3])

In [13]:
gaussians.max_sh_degree

0

In [11]:
gaussians.load_model(args.model_path+"/point_cloud/iteration_30000/point_cloud")

In [16]:
from plyfile import PlyData, PlyElement
plydata = PlyData.read(args.model_path+"/point_cloud/iteration_30000/point_cloud"+".ply")

In [17]:
plydata

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('nx', 'float'), PlyProperty('ny', 'float'), PlyProperty('nz', 'float'), PlyProperty('f_dc_0', 'float'), PlyProperty('f_dc_1', 'float'), PlyProperty('f_dc_2', 'float'), PlyProperty('opacity', 'float'), PlyProperty('scale_0', 'float'), PlyProperty('scale_1', 'float'), PlyProperty('scale_2', 'float'), PlyProperty('rot_0', 'float'), PlyProperty('rot_1', 'float'), PlyProperty('rot_2', 'float'), PlyProperty('rot_3', 'float')), count=2578590, comments=[]),), text=False, byte_order='<', comments=[], obj_info=[])